## LLM Chatbot 실습

#### 라이브러리

In [45]:
import os
from dotenv import load_dotenv
import json
import time
from openai import OpenAI
import google.generativeai as genai
import tiktoken

#### key 가져오기

In [2]:
load_dotenv()

True

In [3]:
gpt_key = os.getenv('GPT_API_KEY')
gemini_key = os.getenv('GOOGLE_API_KEY')
llama_key = os.getenv('HF_TOKEN')

#### system prompt

In [64]:
system_prompt = """
    당신은 친절한 맞춤법 검사 선생님입니다.
    다양한 언어로 쓰여진 문장이 입력되면 입력된 언어의 문법에 맞게 수정해주세요.
    입력된 문장과 수정된 문장을 모두 출력하되, 어떤 내용이 어떻게 틀렸고, 어떻게 수정되었는지도 각각 알려주세요.
    몇 군데가 수정되었는지도 깔끔하게 숫자로 개수만 알려주세요.
    입력된 언어가 어떤 언어인지에 대한 정보를 제공하고 한국어로 번역도 첨부해주세요.
    하지만 입력된 언어가 한국어라면 번역 문장은 출력하지 마세요.
    만약 입력된 문장에서 맞춤법 오류를 찾지 못한 경우에는 수정된 문장 대신 맞춤법 오류를 찾지 못했다고 함께 출력해주세요.
    만약 입력된 문장의 언어의 문법을 알지 못한다면 수정된 문장 대신 알 수 없는 언어라고 함께 출력해주세요.
    만약 사용자가 언어를 제시하면서 이전에 제시한 언어로 물어본 맞춤법 문장에 대해 물어본다면, 수정된 문장과 한국어로 번역된 문장을 출력해주세요.
    하지만 사용자가 제시한 언어로 된 문장을 이전에 입력한 적이 없다면 입력하신 적 없는 언어라고 알려주세요.
    답변 형식은 간결하고 깔끔하게 출력해주세요.
"""

#### gpt-4o

In [5]:
os.environ['OPENAI_API_KEY'] = gpt_key
openai = OpenAI()

In [53]:
gpt_path = './data/gpt_history.json'

def gpt_load_history():
    if os.path.exists(gpt_path):
        with open(gpt_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    return [{'role': 'system', 'content': system_prompt}]

def gpt_save_history(messages):
    with open(gpt_path, 'w', encoding='utf-8') as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

def gpt_get(messages, temperature=0):
    response = openai.chat.completions.create(
        model = 'gpt-4o',
        messages = messages,
        temperature = temperature,
    )
    return response.choices[0].message.content

In [54]:
gpt_messages = gpt_load_history()
print('안녕하세요! 맞춤법 검사를 도와드리겠습니다.\n')
token = tiktoken.get_encoding('cl100k_base')

while True:
    gpt_input = input('[사용자]: ')
    print(f'[사용자]: {gpt_input}\n')
    if gpt_input.lower() in ['exit', 'quit', '종료', '나가기']:
        print('대화를 종료합니다.')
        gpt_save_history(gpt_messages)
        break
    gpt_messages.append({'role': 'user', 'content': gpt_input})
    start_time = time.time()
    gpt_response = gpt_get(gpt_messages)
    gpt_messages.append({'role': 'assistant', 'content': gpt_response})
    print(f'[챗봇]: {gpt_response}\n')
    print(f'토큰 수: {len(token.encode(gpt_response))}')
    end_time = time.time()
    print(f'답변 생성 시간: {end_time-start_time:.2f}초\n')

안녕하세요! 맞춤법 검사를 도와드리겠습니다.

[사용자]: 이렇거 하고 십지 안고, 해서도 않되는 일이야.

[챗봇]: 입력된 문장: 이렇거 하고 십지 안고, 해서도 않되는 일이야.
수정된 문장: 이렇게 하고 싶지 않고, 해서도 안 되는 일이야.
틀린 내용: 
1. "이렇거"는 "이렇게"로 수정되었습니다.
2. "십지"는 "싶지"로 수정되었습니다.
3. "안고"는 "않고"로 수정되었습니다.
4. "않되는"은 "안 되는"으로 수정되었습니다.
수정된 개수: 4
번역: 제공하지 않음 (한국어 문장)

토큰 수: 171
답변 생성 시간: 2.33초

[사용자]: Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.

[챗봇]: 입력된 문장: Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.
수정된 문장: Hier, je suis allé à l'école avec mon ami. Nous avons étudié beaucoup.
틀린 내용:
1. "je vais"는 과거형 "je suis allé"로 수정되었습니다.
2. "mon amis"는 단수형 "mon ami"로 수정되었습니다.
3. "etudier"는 과거분사 "étudié"로 수정되었습니다.
수정된 개수: 3
언어: 프랑스어
번역: 어제, 나는 내 친구와 함께 학교에 갔다. 우리는 많이 공부했다.

토큰 수: 191
답변 생성 시간: 2.95초

[사용자]: Ayer, yo y mi amigo fueron a la tienda. Compramos café y comimos lo.

[챗봇]: 입력된 문장: Ayer, yo y mi amigo fueron a la tienda. Compramos café y comimos lo.
수정된 문장: Ayer, mi amigo y yo fuimos a la tienda. Compramos café y lo bebimos.
틀린 내

In [57]:
gpt_messages = gpt_load_history()
print('안녕하세요! 맞춤법 검사를 도와드리겠습니다.\n')

while True:
    gpt_input = input('[사용자]: ')
    print(f'[사용자]: {gpt_input}\n')
    if gpt_input.lower() in ['exit', 'quit', '종료', '나가기']:
        print('대화를 종료합니다.')
        gpt_save_history(gpt_messages)
        break
    gpt_messages.append({'role': 'user', 'content': gpt_input})
    start_time = time.time()
    gpt_response = gpt_get(gpt_messages)
    gpt_messages.append({'role': 'assistant', 'content': gpt_response})
    print(f'[챗봇]: {gpt_response}\n')
    print(f'토큰 수: {len(token.encode(gpt_response))}')
    end_time = time.time()
    print(f'답변 생성 시간: {end_time-start_time:.2f}초\n')

안녕하세요! 맞춤법 검사를 도와드리겠습니다.

[사용자]: 혹시 내가 아까 스페인어로 된 문장을 입력했었나?

[챗봇]: 네, 이전에 스페인어로 된 문장을 입력하셨습니다.

수정된 문장: Ayer, mi amigo y yo fuimos a la tienda. Compramos café y lo bebimos.
번역: 어제, 내 친구와 나는 가게에 갔다. 우리는 커피를 사고 그것을 마셨다.

토큰 수: 95
답변 생성 시간: 1.56초

[사용자]: 아까 이탈리어로 물어본 문장 뭐였더라?

[챗봇]: 입력하신 적 없는 언어입니다.

토큰 수: 13
답변 생성 시간: 0.85초

[사용자]: 아까 물어본 스웨덴어 문장에 대해 알려줄 수 있어?

[챗봇]: 네, 이전에 스웨덴어로 입력하신 문장은 다음과 같습니다.

입력된 문장: I går, min vän och jag gick till butiken. Vi köpte kaffe och drack det.
수정된 문장: 맞춤법 오류를 찾지 못했습니다.
번역: 어제, 내 친구와 나는 가게에 갔다. 우리는 커피를 사고 그것을 마셨다.

토큰 수: 122
답변 생성 시간: 1.75초

[사용자]: 종료

대화를 종료합니다.


#### gpt-4o-mini

In [19]:
gptmini_path = './data/gptmini_history.json'

def gptmini_load_history():
    if os.path.exists(gptmini_path):
        with open(gptmini_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    return [{'role': 'system', 'content': system_prompt}]

def gptmini_save_history(messages):
    with open(gptmini_path, 'w', encoding='utf-8') as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

def gptmini_get(messages, temperature=0):
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        temperature = temperature,
    )
    return response.choices[0].message.content

In [59]:
gptmini_messages = gptmini_load_history()
print('안녕하세요! 맞춤법 검사를 도와드리겠습니다.\n')

while True:
    gptmini_input = input('[사용자]: ')
    print(f'[사용자]: {gptmini_input}\n')
    if gptmini_input.lower() in ['exit', 'quit', '종료', '나가기']:
        print('대화를 종료합니다.\n')
        gptmini_save_history(gptmini_messages)
        break
    gptmini_messages.append({'role': 'user', 'content': gptmini_input})
    start_time = time.time()
    gptmini_response = gpt_get(gptmini_messages)
    gptmini_messages.append({'role': 'assistant', 'content': gptmini_response})
    print(f'[챗봇]: {gptmini_response}\n')
    print(f'토큰 수: {len(token.encode(gptmini_response))}')
    end_time = time.time()
    print(f'답변 생성 시간: {end_time-start_time:.2f}초\n')

안녕하세요! 맞춤법 검사를 도와드리겠습니다.

[사용자]: 이렇거 하고 십지 안고, 해서도 않되는 일이야.

[챗봇]: 입력된 문장: 이렇거 하고 십지 안고, 해서도 않되는 일이야.
수정된 문장: 이렇게 하고 싶지 않고, 해서도 안 되는 일이야.
틀린 내용 및 수정 사항:
1. "이렇거" -> "이렇게": '이렇거'는 잘못된 표현이며, '이렇게'가 올바른 표현입니다.
2. "십지" -> "싶지": '십지'는 잘못된 표현이며, '싶지'가 올바른 표현입니다.
3. "안고" -> "않고": '안고'는 잘못된 표현이며, '않고'가 올바른 표현입니다.
4. "않되는" -> "안 되는": '않되는'은 잘못된 표현이며, '안 되는'이 올바른 표현입니다.
수정된 개수: 4개

토큰 수: 283
답변 생성 시간: 3.53초

[사용자]: Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.

[챗봇]: 입력된 문장: Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.
수정된 문장: Hier, je suis allé à l'école avec mon ami. Nous avons étudié beaucoup.
틀린 내용 및 수정 사항:
1. "je vais" -> "je suis allé": 'vais'는 현재 시제이며, 'suis allé'가 과거 시제에 맞습니다.
2. "mon amis" -> "mon ami": 'amis'는 복수형이며, 'ami'가 단수형에 맞습니다.
3. "etudier" -> "étudié": 'etudier'는 부정사이며, 'étudié'가 과거 분사에 맞습니다.
수정된 개수: 3개
언어: 프랑스어
번역: 어제, 나는 내 친구와 함께 학교에 갔다. 우리는 많이 공부했다.

토큰 수: 252
답변 생성 시간: 3.65초

[사용자]: Ayer, yo y mi amigo fueron a la tienda. Compram

In [60]:
gptmini_messages = gptmini_load_history()
print('안녕하세요! 맞춤법 검사를 도와드리겠습니다.\n')

while True:
    gptmini_input = input('[사용자]: ')
    print(f'[사용자]: {gptmini_input}\n')
    if gptmini_input.lower() in ['exit', 'quit', '종료', '나가기']:
        print('대화를 종료합니다.\n')
        gptmini_save_history(gptmini_messages)
        break
    gptmini_messages.append({'role': 'user', 'content': gptmini_input})
    start_time = time.time()
    gptmini_response = gpt_get(gptmini_messages)
    gptmini_messages.append({'role': 'assistant', 'content': gptmini_response})
    print(f'[챗봇]: {gptmini_response}\n')
    print(f'토큰 수: {len(token.encode(gptmini_response))}')
    end_time = time.time()
    print(f'답변 생성 시간: {end_time-start_time:.2f}초\n')

안녕하세요! 맞춤법 검사를 도와드리겠습니다.



[사용자]: 혹시 내가 아까 스페인어로 된 문장을 입력했었나?

[챗봇]: 네, 이전에 스페인어로 된 문장을 입력하셨습니다.

토큰 수: 24
답변 생성 시간: 0.95초

[사용자]: 아까 이탈리아어로 물어본 문장 뭐였더라?

[챗봇]: 입력하신 적 없는 언어입니다.

토큰 수: 13
답변 생성 시간: 0.77초

[사용자]: 아까 물어본 스웨덴어 문장에 대해 알려줄 수 있어?

[챗봇]: 수정된 문장: I går, min vän och jag gick till butiken. Vi köpte kaffe och drack det.
번역: 어제, 내 친구와 나는 가게에 갔다. 우리는 커피를 사고 그것을 마셨다.

토큰 수: 72
답변 생성 시간: 1.28초

[사용자]: 종료

대화를 종료합니다.



#### gemini

In [23]:
genai.configure(api_key=gemini_key)

In [61]:
gemini_path = './data/gemini_history.json'

def gemini_load_history():
    if os.path.exists(gemini_path):
        with open(gemini_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    return []

def gemini_save_history(messages):
    with open(gemini_path, 'w', encoding='utf-8') as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

def gemini_get(messages, temperature=0):
    gemini_config = genai.GenerationConfig(
        temperature = temperature,
    )
    gemini = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_prompt, generation_config=gemini_config)
    response = gemini.generate_content(messages)
    token_count = gemini.count_tokens(response.text)
    return response, token_count

In [62]:
gemini_messages = gemini_load_history()
print('안녕하세요! 맞춤법 검사를 도와드리겠습니다.\n')

while True:
    gemini_input = input('[사용자]: ')
    print(f'[사용자]: {gemini_input}\n')
    if gemini_input.lower() in ['exit', 'quit', '종료', '나가기']:
        print('대화를 종료합니다.\n')
        gemini_save_history(gemini_messages)
        break
    gemini_messages.append({'role': 'user', 'parts': [gemini_input]})
    start_time = time.time()
    gemini_response, token_count = gemini_get(gemini_messages)
    gemini_messages.append({'role': 'model', 'parts': [gemini_response.candidates[0].content.parts[0].text]})
    print(f'[챗봇]: {gemini_response.text}\n')
    print(f'토큰 수: {token_count}')
    end_time = time.time()
    print(f'답변 생성 시간: {end_time-start_time:.2f}초\n')

안녕하세요! 맞춤법 검사를 도와드리겠습니다.

[사용자]: 이렇거 하고 십지 안고, 해서도 않되는 일이야.

[챗봇]: **입력된 문장:** 이렇거 하고 십지 안고, 해서도 않되는 일이야.

**언어:** 한국어

**수정된 문장:** 이렇게 하고 싶지 않고, 해서도 안 되는 일이야.

**오류 수정 내역:**

1. "이렇거"를 "이렇게"로 수정했습니다.  잘못된 어미 사용을 바로잡았습니다.
2. "십지"를 "싶지"로 수정했습니다. 잘못된 표기를 바로잡았습니다.
3. "안고"를 "않고"로 수정했습니다.  '않고'는 부정의 의미를 나타내는 조사이므로 '안고'보다 적절합니다.
4. "않되는"을 "안 되는"으로 수정했습니다.  '안 되는'은 '되지 않는'의 준말로, '않되는'은 잘못된 표현입니다.


**총 수정 횟수:** 4


토큰 수: total_tokens: 603

답변 생성 시간: 3.05초

[사용자]: Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.

[챗봇]: **입력된 문장:** Hier, je vais à l'école avec mon amis. Nous avons etudier beaucoup.

**언어:** 프랑스어

**수정된 문장:** Hier, je suis allé(e) à l'école avec mon ami. Nous avons beaucoup étudié.

**오류 수정 내역:**

1. **"vais"를 "suis allé(e)"로 수정:**  "vais"는 현재 시제 동사 "aller"(가다)의 1인칭 단수 현재형입니다.  하지만 문맥상 어제 일을 이야기하므로 과거 시제가 필요합니다. 따라서 과거분사 "allé" (남성) 또는 "allée" (여성)를 사용하여 과거 시제를 표현했습니다.  화자의 성별에 따라 "allé" 또는 "allée"를 선택해야 합니다.
2. **"amis"를 "ami"로 수정:** "amis"는 "친구들"을 의미하는 복수형입니

In [63]:
gemini_messages = gemini_load_history()
print('안녕하세요! 맞춤법 검사를 도와드리겠습니다.\n')

while True:
    gemini_input = input('[사용자]: ')
    print(f'[사용자]: {gemini_input}\n')
    if gemini_input.lower() in ['exit', 'quit', '종료', '나가기']:
        print('대화를 종료합니다.\n')
        gemini_save_history(gemini_messages)
        break
    gemini_messages.append({'role': 'user', 'parts': [gemini_input]})
    start_time = time.time()
    gemini_response, token_count = gemini_get(gemini_messages)
    gemini_messages.append({'role': 'model', 'parts': [gemini_response.candidates[0].content.parts[0].text]})
    print(f'[챗봇]: {gemini_response.text}\n')
    print(f'토큰 수: {token_count}')
    end_time = time.time()
    print(f'답변 생성 시간: {end_time-start_time:.2f}초\n')

안녕하세요! 맞춤법 검사를 도와드리겠습니다.

[사용자]: 혹시 아까 내가 스페인어로 된 문장을 입력했었나?

[챗봇]: 네, 아까 스페인어 문장을 입력하셨습니다.  문장은 "Ayer, yo y mi amigo fueron a la tienda. Compramos café y comimos lo." 였고,  수정된 문장은 "Ayer, mi amigo y yo fuimos a la tienda. Compramos café y lo comimos."였습니다.  한국어 번역은 "어제 친구와 나는 가게에 갔다. 커피를 사서 그것을 먹었다." 입니다.


토큰 수: total_tokens: 483

답변 생성 시간: 2.09초

[사용자]: 아까 이탈리아로 물어본 문장 뭐였더라?

[챗봇]: 이탈리아어로 질문하신 문장은 입력하신 적이 없습니다.


토큰 수: total_tokens: 397

답변 생성 시간: 2.59초

[사용자]: 아까 물어본 스웨덴어 문장에 대해 알려줄 수 있어?

[챗봇]: 네, 아까 스웨덴어로 질문하신 문장은 "I går, min vän och jag gick till butiken. Vi köpte kaffe och drack det." 이었습니다. 수정된 문장은 "I går gick min vän och jag till butiken. Vi köpte kaffe och drack det." 이었고,  한국어 번역은 "어제 친구와 나는 가게에 갔다. 우리는 커피를 사서 마셨다." 입니다.  수정 사항은 어순을 자연스럽게 바꾼 것이었습니다.


토큰 수: total_tokens: 500

답변 생성 시간: 1.67초

[사용자]: 종료

대화를 종료합니다.

